In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Creating the dataframe Neighborhoods of Toronto

In [16]:
URL="https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969"

In [17]:
response = requests.get(URL)

web_page = response.text

soup= BeautifulSoup(web_page, "html.parser")

In [18]:
table_contents=[]
table=soup.find('table')

In [20]:
cell = {"Postal Code":[],"Borough":[],"Neighborhood":[]}
for row in table.findAll('tr')[1:]:
    x=row.text.replace("\n\n",",").replace("\n","").split(",")
    if "Not assigned" in x:
        pass
    else:
        cell['Postal Code'].append(x[0])
        cell['Borough'].append(x[1])
        cell['Neighborhood'].append(x[2:])
table_contents.append(cell)

Dataframe with Postal Code, Borough and Neighborhood

In [21]:
df=pd.DataFrame(cell)
df['Neighborhood']=df['Neighborhood'].apply(', '.join)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [22]:
df.shape

(103, 3)

Uploading the csv file that has the geographical coordinates of each postal code

In [23]:
coor=pd.read_csv("Geospatial_Coordinates.csv")

In [24]:
coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
df_merged = pd.merge(df, coor)

Merging two dataframes

In [37]:
df_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South ...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, ...",43.636258,-79.498509


Exploring and clustering Toronto neighborhoods

In [34]:
from geopy.geocoders import Nominatim
import folium
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [35]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Borough'], df_merged['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto